Models Installation:
Please note to restart your runtime for these to take effect

In [1]:
!python -m spacy download en
!pip install spacy-transformers[cuda100]==0.5.1
!python -m spacy download en_trf_bertbaseuncased_lg
!python -m spacy download en_trf_robertabase_lg
!python -m spacy download en_trf_distilbertbaseuncased_lg


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 10.6MB 8.2MB/s 
     |████████████████████████████████| 296kB 54.9MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 491kB 45.4MB/s 
     |████████████████████████████████| 337.6MB 49kB/s 
     |████████████████████████████████| 3.7MB 35.2MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 122kB 60.8MB/s 
     |████████████████████████████████| 2.2MB 45.3MB/s 
     |████████████████████████████████| 870kB 54.3MB/s 
     |████████████████████████████████| 1.0MB 44.1MB/s 
  Created wheel for spacy-transformers: filename=spacy_

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
     |████████████████████████████████| 405.8MB 1.2MB/s 
  Created wheel for en-trf-bertbaseuncased-lg: filename=en_trf_bertbaseuncased_lg-2.2.0-cp36-none-any.whl size=405819945 sha256=c372f55f8909171a75b6f40d06e404ad4e296176d4bbb343a5c1fbf67e05e455
  Stored in directory: /tmp/pip-ephem-wheel-cache-9xecsi5r/wheels/f6/60/8c/c6f517ef9729972f1be15c3aab4b93e7ec9fbeb71d072a84de
Successfully built en-trf-bertbaseuncased-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_trf_bertbaseuncased_lg')
⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself in

One time step

In [1]:
#One time step
!mkdir /content/googleUSE
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC /content/googleUSE


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
./
./tfhub_module.pb
./variables/
./variables/variables.data-00000-of-00001
 91  745M   91  681M    0     0  54.6M      0  0:00:13  0:00:12  0:00:01 49.6M./variables/variables.index
./assets/
./saved_model.pb
100  745M  100  745M    0     0  55.2M      0  0:00:13  0:00:13 --:--:-- 54.4M


In [0]:
class QuerySnippet:

    def __init__(self, query, snippet, similarity):
        self.query = query
        self.snippet = snippet
        self.similarity = similarity

    
    def __gt__(self, other):
        return self.similarity > other.similarity

    def __lt__(self, other):
        return self.similarity < other.similarity


In [14]:
import tensorflow_hub as hub
import csv 
import numpy as np
import heapq
import tensorflow

modelHeapDict2 = {}
model = 'Google_USE'

def googleUSEEmbedding(modelName):
    with tensorflow.Graph().as_default():
        researchStr = tensorflow.placeholder(tensorflow.string)
        hubModuleFn = hub.Module(modelName)
        reserachEmbedRes = hubModuleFn(researchStr)
        s = tensorflow.train.MonitoredSession()
    return lambda x: s.run(reserachEmbedRes, {researchStr: x})
researchEmbedding = googleUSEEmbedding('/content/googleUSE/')

def similarSentencesWIthGoogleUSE(researchPaper, queriesList, highestScoredNSnippets):
  print('Inside')
  with open(researchPaper) as researchPaperCSV:
    researchPaperReader = csv.reader(researchPaperCSV)
    print(researchPaperReader)
    score_max_heap = [] 
    for query in queriesList:
      #print(query)
      embeddedQuery = researchEmbedding([query])
      #print(embeddedQuery)
      for snippet in researchPaperReader:
          if('<EOS>' not in snippet):
            snippetStr = " "
            snippetStr = ' '.join([str(elem) for elem in snippet])
            #print('snippetStr:', snippetStr)
            embeddedSnippet = researchEmbedding([snippetStr])
            #print(embeddedSnippet)
            qs = QuerySnippet(query, snippet,  np.inner(embeddedQuery, embeddedSnippet))
            print(qs.query)
            print(qs.snippet)
            print(qs.similarity)
            if len(score_max_heap) < highestScoredNSnippets or qs.similarity > score_max_heap[0].similarity:
              if len(score_max_heap) == highestScoredNSnippets: heapq.heappop(score_max_heap)
              heapq.heappush( score_max_heap, qs )
      modelHeapDict2[model] =  score_max_heap       
  return  modelHeapDict2  

#similarSentencesWIthGoogleUSE('/content/large_batch_optimization_for_deep_learning_training_bert_in_76_minutes_1.csv',["Which is the best model"],10)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [15]:
'''for h in modelHeapDict2:
  print(h)
  print(modelHeapDict2[h])
  while modelHeapDict2[h]:
    qs = heapq.heappop(modelHeapDict2[h])
    print(qs.snippet)
    print(qs.similarity)  
'''

'for h in modelHeapDict2:\n  print(h)\n  print(modelHeapDict2[h])\n  while modelHeapDict2[h]:\n    qs = heapq.heappop(modelHeapDict2[h])\n    print(qs.snippet)\n    print(qs.similarity)  \n'

Pass the snippet csv file, query list and number of results desired.


In [16]:
useHeapDict=similarSentencesWIthGoogleUSE('/content/graph_inference_learning_for_semi_supervised_classification_3.csv',["Which is the best model"],10)


Inside
Which is the best model
['Published as a conference paper at ICLR 2020  GRAPH INFERENCE LEARNING FOR SEMI-SUPERVISED CLASSIFICATION  Chunyan Xu', ' Zhen Cui∗', ' Xiaobin Hong', ' Tong Zhang', ' and Jian Yang School of Computer Science and Engineering', ' Nanjing University of Science and Technology', ' Nanjing', ' China {cyx', 'zhen.cui', 'xbhong', 'tong.zhang', 'csjyang}@njust.edu.cn  Wei Liu Tencent AI Lab', ' China wl2223@columbia.edu  ABSTRACT  In this work', ' we address semi-supervised classification of graph data', ' where the categories of those unlabeled nodes are inferred from labeled nodes as well as graph structures.Recent works often solve this problem via advanced graph convolution in a conventionally supervised manner', ' but the performance could degrade significantly when labeled data is scarce.To this end', ' we propose a Graph Inference Learning (GIL) framework to boost the performance of semi- supervised node classification by learning the inference of node l

In [17]:
'''modelResultFile = open("/content/Model.txt", "a")

for model in useHeapDict:
  modelResultFile.write(model)
  modelResultFile.write("\n")
  modelHeap = useHeapDict[model]
  while modelHeap: 
    qs = heapq.heappop(modelHeap)
    modelResultFile.write(qs.query)
    modelResultFile.write("\n")
    modelResultFile.write(' '.join([str(elem) for elem in qs.snippet]))
    modelResultFile.write("\n")
    modelResultFile.write(str(qs.similarity))
    modelResultFile.write("\n")
    

modelResultFile.close()
'''

'modelResultFile = open("/content/Model.txt", "a")\n\nfor model in useHeapDict:\n  modelResultFile.write(model)\n  modelResultFile.write("\n")\n  modelHeap = useHeapDict[model]\n  while modelHeap: \n    qs = heapq.heappop(modelHeap)\n    modelResultFile.write(qs.query)\n    modelResultFile.write("\n")\n    modelResultFile.write(\' \'.join([str(elem) for elem in qs.snippet]))\n    modelResultFile.write("\n")\n    modelResultFile.write(str(qs.similarity))\n    modelResultFile.write("\n")\n    \n\nmodelResultFile.close()\n'

Other Models


In [0]:
import spacy
import csv
import heapq

In [19]:

def compareDifferentLanguageModels(researchPaper,queriesList,highestScoredNSnippets = 10):
  spacy_model_list = ['en_trf_bertbaseuncased_lg','en_trf_robertabase_lg','en_trf_distilbertbaseuncased_lg']
  #queriesList= ["Which model is the best"]
  modelHeapDict = {}
  #model_bert = 'en_trf_bertbaseuncased_lg'
  
  
  #with open(researchPaper) as researchPaperCSV:
  #researchPaperReader = csv.reader(researchPaperCSV)
  #print(researchPaperCSV)
  #For every model , iterate
  for model in spacy_model_list:
    researchPpr_NLP = spacy.load(model)

    with open(researchPaper) as researchPaperCSV:
      researchPaperReader = csv.reader(researchPaperCSV)
      score_max_heap = [] 
      for query in queriesList:
        queryObj = researchPpr_NLP(query)
        print(queryObj)
        for snippet in researchPaperReader:
            #print(snippet)
            #EOS tag ignore
            if('<EOS>' not in snippet):
              #print(snippet)
              #snippet = 'In Advances in neural information processing systems pp 693–701 2011.'
              snippetStr = " "
              snippetStr = ' '.join([str(elem) for elem in snippet])
              #print(snippetStr)
              snippetObj = researchPpr_NLP(snippetStr)
              #print(snippetObj)
              qs = QuerySnippet(query, snippet,  queryObj.similarity(snippetObj))
              print(model)
              #print(qs.query)
              print(qs.snippet)
              print(qs.similarity)
              #heap logic
              if len(score_max_heap) < highestScoredNSnippets or qs.similarity > score_max_heap[0].similarity:
                if len(score_max_heap) == highestScoredNSnippets: heapq.heappop(score_max_heap)
                heapq.heappush( score_max_heap, qs )
    modelHeapDict[model] =  score_max_heap
    print(model)
    print(len(score_max_heap))            
  return  modelHeapDict         
'''
modelHeapDict=compareDifferentLanguageModels('/content/large_batch_optimization_for_deep_learning_training_bert_in_76_minutes_1.csv')
print(modelHeapDict)
for h in modelHeapDict:
  print(h)
  print(modelHeapDict[h])
  while modelHeapDict[h]:
    qs = heapq.heappop(modelHeapDict[h])
    print(qs.snippet)
    print(qs.similarity)
'''
  




"\nmodelHeapDict=compareDifferentLanguageModels('/content/large_batch_optimization_for_deep_learning_training_bert_in_76_minutes_1.csv')\nprint(modelHeapDict)\nfor h in modelHeapDict:\n  print(h)\n  print(modelHeapDict[h])\n  while modelHeapDict[h]:\n    qs = heapq.heappop(modelHeapDict[h])\n    print(qs.snippet)\n    print(qs.similarity)\n"

Pass The file and  Snippet arraylist below

In [20]:
modelHeapDict=compareDifferentLanguageModels('/content/graph_inference_learning_for_semi_supervised_classification_3.csv',["Which is the best model"])


Which is the best model
en_trf_bertbaseuncased_lg
['Published as a conference paper at ICLR 2020  GRAPH INFERENCE LEARNING FOR SEMI-SUPERVISED CLASSIFICATION  Chunyan Xu', ' Zhen Cui∗', ' Xiaobin Hong', ' Tong Zhang', ' and Jian Yang School of Computer Science and Engineering', ' Nanjing University of Science and Technology', ' Nanjing', ' China {cyx', 'zhen.cui', 'xbhong', 'tong.zhang', 'csjyang}@njust.edu.cn  Wei Liu Tencent AI Lab', ' China wl2223@columbia.edu  ABSTRACT  In this work', ' we address semi-supervised classification of graph data', ' where the categories of those unlabeled nodes are inferred from labeled nodes as well as graph structures.Recent works often solve this problem via advanced graph convolution in a conventionally supervised manner', ' but the performance could degrade significantly when labeled data is scarce.To this end', ' we propose a Graph Inference Learning (GIL) framework to boost the performance of semi- supervised node classification by learning the 

Pass the filename with path to save the results 

In [21]:

modelResultFile = open("/content/ModelComparison.tsv", "a")

for model in modelHeapDict:
  modelResultFile.write(model)
  modelResultFile.write("\n")
  modelHeap = modelHeapDict[model]
  while modelHeap:
    qs = heapq.heappop(modelHeap)
    modelResultFile.write(model)
    modelResultFile.write("\t")
    modelResultFile.write(qs.query)
    modelResultFile.write("\t")
    modelResultFile.write(' '.join([str(elem) for elem in qs.snippet]))
    modelResultFile.write("\t")
    modelResultFile.write(str(qs.similarity))
    print(qs.similarity)
    modelResultFile.write("\n")

for model in useHeapDict:
  modelResultFile.write(model)
  modelResultFile.write("\n")
  modelHeap = useHeapDict[model]
  while modelHeap: 
    qs = heapq.heappop(modelHeap)
    modelResultFile.write(model)
    modelResultFile.write("\t")
    modelResultFile.write(qs.query)
    modelResultFile.write("\t")
    modelResultFile.write(' '.join([str(elem) for elem in qs.snippet]))
    modelResultFile.write("\t")
    modelResultFile.write(str(qs.similarity))
    modelResultFile.write("\n")
    

modelResultFile.close()

0.6203900028208551
0.6204926895437002
0.6256185225063298
0.6312060808024065
0.6407311347778408
0.6436403041153803
0.6456181124170579
0.6497621031393012
0.6562072824702831
0.6608844701429274
0.9614559044744097
0.9614585965982105
0.9617157319046222
0.9617517456508072
0.9620299779231642
0.9625336887797311
0.9627932951627081
0.9642085722761409
0.9658407911474166
0.9692078493296902
0.7480377612141431
0.7490114852212207
0.749372092411089
0.7515062052637874
0.7518548587494518
0.7573500433857987
0.7641087296994077
0.7663656172437061
0.7689564858620309
0.7819935650942139
